# Imports

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
import google.colab
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import scipy.stats as stats
from sklearn.feature_selection import f_classif

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
music_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/Music/cleaned_df.csv")

In [5]:
music_df

,artist,song,emotion,variance,genre,release date,key,tempo,loudness,explicit,...,speechiness,liveness,acousticness,instrumentalness,speechiness_log,instrumentalness_bin,mode_int,key_root,cos_pitch,sin_pitch
0,ABBA,She's My Kind Of Girl,joy,0.447619,pop,2014,F Maj,128,-6.00,No,...,3,31,7,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
1,ABBA,"Andante, Andante",love,0.202222,pop,1980,A# Maj,102,-10.72,No,...,2,7,68,0,1.098612,0,1,10,5.000000e-01,-8.660254e-01
2,ABBA,As Good As New,sadness,0.300881,pop,1979,E Maj,139,-5.70,No,...,3,8,20,2,1.386294,1,1,4,-5.000000e-01,8.660254e-01
3,ABBA,Bang,joy,0.355000,pop,1975,F Maj,132,-3.00,No,...,3,32,3,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
4,ABBA,Bang-A-Boomerang,joy,0.355000,pop,1975,F Maj,132,-3.00,No,...,3,32,3,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236964,(Eri Sasaki),Gate Of Steiner,sadness,0.833514,other,2018,D min,148,-6.44,No,...,4,14,20,0,1.609438,0,0,2,5.000000e-01,8.660254e-01
236965,(Elena Tsagrinou),El Diablo,love,0.833514,other,2021,A min,114,-7.78,No,...,13,84,0,0,2.639057,0,0,9,-1.836970e-16,-1.000000e+00
236966,(Dima Bilan),Believe,joy,0.833514,other,2009,C min,134,-6.72,No,...,5,22,5,0,1.791759,0,0,0,1.000000e+00,0.000000e+00
236967,(Ani Lorak),Shady Lady,joy,0.833514,other,2009,F# min,128,-13.00,No,...,5,7,5,0,1.791759,0,0,6,-1.000000e+00,1.224647e-16


In [9]:
all_files = glob.glob("/content/drive/MyDrive/Colab Notebooks/project/Music/users history/*.csv")
history_list = [pd.read_csv(f) for f in all_files]
history_df = pd.concat(history_list, ignore_index=True)

In [10]:
history_df

,user_id,song_id,artist,mood_before,mood_after,mood_improved,timestamp
0,user_1,Im a Man,Bo Diddley,surprise,surprise,0,2025-04-12 10:01:27
1,user_1,Da Nex Niguz,Onyx,fear,fear,0,2025-04-22 20:53:27
2,user_1,The Haunting Somewhere in Time,Kamelot,fear,fear,0,2025-04-26 18:14:27
3,user_1,It Started With a Kiss,Hot Chocolate,surprise,surprise,0,2025-04-13 13:56:27
4,user_1,My Only Shawty,Demi Lovato,joy,joy,0,2025-04-18 00:13:27
...,...,...,...,...,...,...,...
9995,user_100,Dumbfoundead vs. Tantrum,Grind Time Now,anger,anger,0,2025-04-11 12:17:16
9996,user_100,I'm Not Giving Up,Gloria Estefan,sadness,sadness,0,2025-04-10 12:19:16
9997,user_100,Ghoulish,Lil Wayne,fear,fear,0,2025-04-22 08:45:16
9998,user_100,Dream,The Pied Pipers,love,love,0,2025-05-02 21:31:16


In [8]:
import pandas as pd
import glob
feature_columns = ['tempo','loudness','energy','danceability','positiveness',
                     'speechiness_log','liveness','acousticness','instrumentalness_bin','cos_pitch', 'sin_pitch', 'mode_int']
# Step 1: Load + merge all user histories


# Step 2: Join with main song features
song_features = df.set_index('song').loc[history_df['song_id']].reset_index()
full_df = pd.concat([history_df.reset_index(drop=True), song_features[feature_columns]], axis=1)

# Step 3: Train ML model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = full_df[feature_columns]
y = full_df['mood_improved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Evaluate + plot feature importance
import matplotlib.pyplot as plt

importances = clf.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(feature_columns, importances)
plt.xlabel("Importance")
plt.title("🎯 Song Attribute Importance for Mood Improvement")
plt.gca().invert_yaxis()
plt.show()


ValueError: Input y contains NaN.